In [1]:
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
from collections import Counter
from itertools import chain,combinations

In [2]:
# https://towardsdatascience.com/topic-modeling-with-bert-779f7db187e6

from collect_data import load_cache
from tokenizer import load_tokens

tokens = load_tokens('/data/blockchain-interoperability/blockchain-social-media/analysis/twitter/tokens/text')
# data = load_cache('/data/blockchain-interoperability/blockchain-social-media/analysis/twitter/snapshots')

loading tokens...:   0%|          | 0/47 [00:00<?, ?it/s]

In [3]:
joined_tokens = list(map(lambda x: ' '.join(x),tokens))

In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2',device='cuda')
embeddings = model.encode(joined_tokens[:1000], show_progress_bar=True)

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [9]:
embeddings.shape

(1000, 384)

In [8]:
# dimensionality reduction

import umap
umap_embeddings = umap.UMAP(n_neighbors=15, 
                            n_components=5, 
                            metric='cosine').fit_transform(embeddings)

/data/blockchain-interoperability/blockchain-social-media/blockchain-sns-env/lib64/python3.6/site-packages/numba/np/ufunc/parallel.py:365: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 6103. The TBB threading layer is disabled.
  warnings.warn(problem)


In [12]:
umap_embeddings.shape

(1000, 5)

In [ ]:
# clustering the reduced dimensions

import hdbscan
cluster = hdbscan.HDBSCAN(min_cluster_size=15,
                          metric='euclidean',                      
                          cluster_selection_method='eom').fit(umap_embeddings)